In [1]:
import numpy as np
import pandas as pd

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

2021-09-30 02:13:30.896149: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-30 02:13:30.896278: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DEBUG = False

train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

if DEBUG:
    train = train[:80*1000]

In [3]:
# def add_features(df):
#     # rewritten calculation of lag features from this notebook: https://www.kaggle.com/patrick0302/add-lag-u-in-as-new-feat
#     # some of ideas from this notebook: https://www.kaggle.com/mst8823/google-brain-lightgbm-baseline
#     df['last_value_u_in'] = df.groupby('breath_id')['u_in'].transform('last')
#     df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
#     df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
#     df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
#     df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
#     df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
#     df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
#     df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
#     df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
#     df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
#     df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
#     df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
#     df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
#     df = df.fillna(0)


#     df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)

#     # max value of u_in and u_out for each breath
#     df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
#     df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')

#     # difference between consequitive values
#     df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
#     df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
#     df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
#     df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
#     # from here: https://www.kaggle.com/yasufuminakama/ventilator-pressure-lstm-starter
#     df.loc[df['time_step'] == 0, 'u_in_diff'] = 0
#     df.loc[df['time_step'] == 0, 'u_out_diff'] = 0

#     # difference between the current value of u_in and the max value within the breath
#     df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
#     df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

#     # OHE
#     df = df.merge(pd.get_dummies(df['R'], prefix='R'), left_index=True, right_index=True).drop(['R'], axis=1)
#     df = df.merge(pd.get_dummies(df['C'], prefix='C'), left_index=True, right_index=True).drop(['C'], axis=1)
#     df = df.merge(pd.get_dummies(df['R__C'], prefix='R__C'), left_index=True, right_index=True).drop(['R__C'], axis=1)

#     # https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/273974
#     df['u_in_cumsum'] = df.groupby(['breath_id'])['u_in'].cumsum()
#     df['time_step_cumsum'] = df.groupby(['breath_id'])['time_step'].cumsum()
#     return df

In [4]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df

train = add_features(train)
test = add_features(test)

In [5]:
train.head()

,id,breath_id,time_step,u_in,u_out,pressure,area,u_in_cumsum,u_in_lag1,u_out_lag1,...,C_50,R__C_20__10,R__C_20__20,R__C_20__50,R__C_50__10,R__C_50__20,R__C_50__50,R__C_5__10,R__C_5__20,R__C_5__50
0,1,1,0.000000,0.083334,0,5.837492,0.000000,0.083334,0.000000,0.0,...,1,0,0,1,0,0,0,0,0,0
1,2,1,0.033652,18.383041,0,5.907794,0.618632,18.466375,0.083334,0.0,...,1,0,0,1,0,0,0,0,0,0
2,3,1,0.067514,22.509278,0,7.876254,2.138333,40.975653,18.383041,0.0,...,1,0,0,1,0,0,0,0,0,0
3,4,1,0.101542,22.808822,0,11.742872,4.454391,63.784476,22.509278,0.0,...,1,0,0,1,0,0,0,0,0,0
4,5,1,0.135756,25.355850,0,12.234987,7.896588,89.140326,22.808822,0.0,...,1,0,0,1,0,0,0,0,0,0


In [6]:
train.shape

(6036000, 53)

In [7]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

In [8]:
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [9]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [10]:
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K
class WarmupExponentialDecay(Callback):
    def __init__(self,lr_base=0.0002,lr_min=0.0,decay=0,warmup_epochs=0):
        self.num_passed_batchs = 0   #一个计数器
        self.warmup_epochs=warmup_epochs  
        self.lr=lr_base #learning_rate_base
        self.lr_min=lr_min #最小的起始学习率,此代码尚未实现
        self.decay=decay  #指数衰减率
        self.steps_per_epoch=0 #也是一个计数器
        
    def on_batch_begin(self, batch, logs=None):
        # params是模型自动传递给Callback的一些参数
        if self.steps_per_epoch==0:
            #防止跑验证集的时候呗更改了
            if self.params['steps'] == None:
                self.steps_per_epoch = np.ceil(1. * self.params['samples'] / self.params['batch_size'])
            else:
                self.steps_per_epoch = self.params['steps']
        if self.num_passed_batchs < self.steps_per_epoch * self.warmup_epochs:
            K.set_value(self.model.optimizer.lr,
                        self.lr*(self.num_passed_batchs + 1) / self.steps_per_epoch / self.warmup_epochs)
        else:
            K.set_value(self.model.optimizer.lr,
                        self.lr*((1-self.decay)**(self.num_passed_batchs-self.steps_per_epoch*self.warmup_epochs)))
        self.num_passed_batchs += 1
        
    def on_epoch_begin(self,epoch,logs=None):
        #用来输出学习率的,可以删除
        print("learning_rate:",K.get_value(self.model.optimizer.lr))

In [11]:
EPOCH = 300
BATCH_SIZE = 1024
NUM_FOLDS = 10

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=2021)
    test_preds = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        model = keras.models.Sequential([
            keras.layers.Input(shape=train.shape[-2:]),
            keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
            keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
#             keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
            keras.layers.Dense(128, activation='selu'),
#             keras.layers.Dropout(0.1),
            keras.layers.Dense(1),
        ])
        model.compile(optimizer="adam", loss="mae")

#         scheduler = ExponentialDecay(1e-3, 40*((len(train)*0.8)/BATCH_SIZE), 1e-5)
#         lr = LearningRateScheduler(scheduler, verbose=1)
        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10, verbose=1)
#         lr = WarmupExponentialDecay(lr_base=1e-3, decay=1e-5, warmup_epochs=30)
        es = EarlyStopping(monitor="val_loss", patience=60, verbose=1, mode="min", restore_best_weights=True)
    
        checkpoint_filepath = f"folds{fold}.hdf5"
        sv = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
            save_weights_only=False, mode='auto', save_freq='epoch',
            options=None
        )

        model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=[lr, es, sv])
        #model.save(f'Fold{fold+1} RNN Weights')
        test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

2021-09-30 02:15:15.950980: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-30 02:15:15.954518: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-09-30 02:15:15.954586: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-30 02:15:15.954618: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3ccfffa1ee28): /proc/driver/nvidia/version does not exist
2021-09-30 02:15:15.958074: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

--------------- > Fold 1 < ---------------


2021-09-30 02:15:38.388887: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1086480000 exceeds 10% of free system memory.


Epoch 1/300
67/67 [==============================] - 55s 423ms/step - loss: 4.0230 - val_loss: 1.1259

Epoch 00001: val_loss improved from inf to 1.12590, saving model to folds0.hdf5
Epoch 2/300
67/67 [==============================] - 9s 142ms/step - loss: 1.0158 - val_loss: 0.6850

Epoch 00002: val_loss improved from 1.12590 to 0.68500, saving model to folds0.hdf5
Epoch 3/300
67/67 [==============================] - 10s 142ms/step - loss: 0.7020 - val_loss: 0.5718

Epoch 00003: val_loss improved from 0.68500 to 0.57180, saving model to folds0.hdf5
Epoch 4/300
67/67 [==============================] - 9s 142ms/step - loss: 0.5841 - val_loss: 0.5558

Epoch 00004: val_loss improved from 0.57180 to 0.55576, saving model to folds0.hdf5
Epoch 5/300
67/67 [==============================] - 9s 142ms/step - loss: 0.5306 - val_loss: 0.5381

Epoch 00005: val_loss improved from 0.55576 to 0.53806, saving model to folds0.hdf5
Epoch 6/300
67/67 [==============================] - 9s 141ms/step - los

2021-09-30 02:44:51.414796: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 804800000 exceeds 10% of free system memory.


...

Epoch 1/300
67/67 [==============================] - 56s 429ms/step - loss: 4.0624 - val_loss: 1.2760

Epoch 00001: val_loss improved from inf to 1.27597, saving model to folds1.hdf5
Epoch 2/300
67/67 [==============================] - 9s 141ms/step - loss: 1.1183 - val_loss: 0.7569

Epoch 00002: val_loss improved from 1.27597 to 0.75691, saving model to folds1.hdf5
Epoch 3/300
67/67 [==============================] - 9s 141ms/step - loss: 0.7393 - val_loss: 0.6914

Epoch 00003: val_loss improved from 0.75691 to 0.69145, saving model to folds1.hdf5
Epoch 4/300
67/67 [==============================] - 9s 141ms/step - loss: 0.6352 - val_loss: 0.5356

Epoch 00004: val_loss improved from 0.69145 to 0.53558, saving model to folds1.hdf5
Epoch 5/300
67/67 [==============================] - 9s 141ms/step - loss: 0.5478 - val_loss: 0.5801

Epoch 00005: val_loss did not improve from 0.53558
Epoch 6/300
67/67 [==============================] - 9s 142ms/step - loss: 0.5143 - val_loss: 0.5244

Epoc

2021-09-30 03:16:37.834751: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 804800000 exceeds 10% of free system memory.


--------------- > Fold 3 < ---------------


2021-09-30 03:17:18.734747: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1086480000 exceeds 10% of free system memory.


Epoch 1/300
67/67 [==============================] - 55s 411ms/step - loss: 3.9154 - val_loss: 1.1166

Epoch 00001: val_loss improved from inf to 1.11658, saving model to folds2.hdf5
Epoch 2/300
67/67 [==============================] - 9s 141ms/step - loss: 0.9747 - val_loss: 0.7662

Epoch 00002: val_loss improved from 1.11658 to 0.76618, saving model to folds2.hdf5
Epoch 3/300
67/67 [==============================] - 9s 141ms/step - loss: 0.6723 - val_loss: 0.6095

Epoch 00003: val_loss improved from 0.76618 to 0.60952, saving model to folds2.hdf5
Epoch 4/300
67/67 [==============================] - 9s 141ms/step - loss: 0.5782 - val_loss: 0.5568

Epoch 00004: val_loss improved from 0.60952 to 0.55685, saving model to folds2.hdf5
Epoch 5/300
67/67 [==============================] - 9s 142ms/step - loss: 0.5179 - val_loss: 0.4787

Epoch 00005: val_loss improved from 0.55685 to 0.47865, saving model to folds2.hdf5
Epoch 6/300
67/67 [==============================] - 10s 142ms/step - los

In [12]:
!ls ./

__notebook__.ipynb  folds2.hdf5  folds5.hdf5  folds8.hdf5
folds0.hdf5	    folds3.hdf5  folds6.hdf5  folds9.hdf5
folds1.hdf5	    folds4.hdf5  folds7.hdf5


In [13]:
submission["pressure"] = sum(test_preds)/NUM_FOLDS
submission.to_csv('submission.csv', index=False)